# Training and Cross-Validating a Predictive Model Using PySpark through the Lens of NHL Expected Goals

### Goal and Overview

This notebook aims to show how to conduct a simple machine learning project from data preparation, training and cross-validation, and choosing a model based on results from a test dataset. It is less focused on developing "the right model" as much as it is focused on showing *how* to choose the correct model to use. 

The sports analytics revolution of the past few years has given rise to new ways of evaluating players. Hockey is not an exception. Fans have been using the NHL's public data to calculate statistics that attempt to more accurately and objectively measure player performance. First came *Corsi*, that is shot attempts made by a team, whether they are a goal, saved by goaltender, miss the net, or are blocked on their way towards the net. A team that has a higher Corsi than its opponents is thought to spend more time on the attack and possessing the puck than its opponent. This indicates better quality play. Similarly, players are also tracked for Corsi statistics. In this context, Corsi refers to shot attempts, for and against a team, that occur when a player is on the ice. A player that consistently has a positive Corsi is thought to have a positive effect on play. 

Let's see an example of how this works: a player X is on the ice for 3 shot attempts for his team and 1 shot attempt against his team. This means that the player is positive on Corsi. This is usually expressed as a percentage, Corsi For Percentage (CF%), where we divide Corsi For by total Corsi with a player on the ice to obtain. In the case of X, CF% is 3/(3+1) = 3/4 = 75%. This player had a very good effect on his team's performance. Conversely, player Y was on the ice for 8 Corsi For and 12 Corsi Against. This translates to CF% of 8/(8+12) = 8/20 = 40%, which is not very good. In the context of the game of hockey, where players regularly change on and off the ice fluidly as the game is played without a clock stoppage, this helps us see which players had a more positive performance on their team's performance since we have many opportunities to compare team performance with and without the player being active in-game.

Such a measure of course ignores many factors that differentiate different shots. It doesn't into account shot distance or the context in which the shot was made. As such, using Corsi alone to evaluate player or team performance gives us an incomplete picture.

This is the role that *expected goals* play. By incorporating more information into our evaluation of team performance, we can get a more accurate picture of how a team or a player played. Expected goals are measured on a shot-by-shot basis, where the xG of a shot is the probability, given the different characteristics of the game at the time of the shot, to be a goal. The higher the xG value, the more likely a shot is to be a goal. We then use aggregate xG values to evaluate a player or a team in much the same way we evaluate them using Corsi. A player with a higher xG recorded for their team than xG against had a positive effect on play while on the ice. Since each shot's factors are mostly independent of each other, we are able to sum xG values because we are adding the the probabilities for independent events.

xG works similarly as before: a player is on the ice for 0.73 xGF and 0.35 xGA. Their xGF% is thus 0.73/(0.73+0.35) = 0.73/1.08 = 67.6%. 

### Model Setup and How I Will Proceed

How will I calculate xG? I can use a model that is calibrated on game events that allows us as much as possible to capture the game situation as a shot is taken. Based on data regarding this shot and many others like it, I can then estimate the likelihood it would have been a goal. Such a model, when trained, can be used to evaluate shots taken when a player is on the ice vs. off the ice, allowing me to estimate shot and team play quality when a player is active in-game. There are several options of models to use. One possibility is obvious from understanding what xG for any shot attempt is: it is the probability that that attempt is a goal. The easiest way to estimate the probability of an event is through logistic regression. 

Another possible approach is to use a tree classifier. In this instance, we can use the proportion of goals that are present at a node after running a datapoint through it as the probability of a goal. For this project, I will in fact be using a random forest classifier. The probability from a random forest is the average of each probability predicted by each tree in the forest. The advantage of a random forest model is that it allows for the possibility of nonlinear interactions between variables that might have a big impact on the probability of a goal.

### Data

The data is obtained from [Moneypuck.com](moneypuck.com) who generously provide their cleaned dataset for anyone to use. Moneypuck scrapes the NHL API and uses game events to construct its dataset, which includes all unblocked shots taken by all players starting with the 2007-08 season. This data runs until the present data. At the time of writing this, the NHL has suspended its 2019-20 season to help in limiting the effects of SARS-COVID-19. We can use this season's data, along with the 2018-19 season as our testing dataset. Data from all seasons before that will be used for training and cross-validation. Once we calibrate our models on the training/CV dataset, we will choose the model that performs best on our test dataset. 

The variables included in the Moneypuck dataset are extensive and capture many details of what was occuring during the game at the time of the shot. They include variables such as the game score, whether either of the nets was empty, how many players were on the ice for each team (i.e. the game situation), how much time has elapsed in the game, and the shot coordinates. I will use several of these variables to estimate my model.

### Libraries/Packages Used

I will use PySpark for data reading and processing, as well as training, cross-validating, and testing models. I use Pandas to obtain column names.

I will import the data, create several variables that we will be using in our model, and then running several models to cross-validate them. After training all models, I will evaluate them on the test dataset and choosing the one that performs best out-of-sample.

In [1]:
import pandas as pd
import numpy as np
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import (IntegerType,
 StringType,DecimalType,StructType,StructField,
 ArrayType,DoubleType,FloatType)
import pyspark.sql.functions as func
from pyspark.ml.feature import (StringIndexer, 
                                OneHotEncoder,
                                OneHotEncoderEstimator, 
                                VectorAssembler)
from pyspark.ml.classification import (LogisticRegression, 
                                       LogisticRegressionModel,
                                      RandomForestClassifier,
                                      RandomForestClassificationModel)
from pyspark.ml import Pipeline
import pyspark.ml.evaluation as evals
from pyspark.ml.evaluation import Evaluator
import pyspark.ml.tuning as tune
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# sc = SparkContext('local[*]')
spark = SparkSession.builder.master('local[*]').config('spark.driver.memory','3g').appName('xG_model').getOrCreate()

### Reading Data and Overview of the Variables

I use Pandas to read the very top of the csv file. I then use Pandas to read the column names and print them for reference. A dictionary of variables and more details is located in the data folder. I then read all columns into a Spark DataFrame as string columns.

In [2]:
data_path = '../data/processed/shots_2007-2018.csv'
data_path_2 = '../data/processed/shots_2019.csv'

sample = pd.read_csv(data_path,nrows=10)
column_names = sample.columns

sample_2 = pd.read_csv(data_path_2,nrows=10)
column_names_2 = sample_2.columns

for ix,x in enumerate(column_names):
    print(ix,x)

schema = StructType([StructField(x,StringType(),True) for x in column_names])
schema_2 = StructType([StructField(x,StringType(),True) for x in column_names_2])

shots_1 = spark.read.csv(data_path,schema=schema,
                       enforceSchema=True,header=True,ignoreLeadingWhiteSpace=True,
                       ignoreTrailingWhiteSpace=True)

shots_2 = spark.read.csv(data_path_2,schema=schema_2,
                         enforceSchema=True,header=True,
                        ignoreLeadingWhiteSpace=True,
                        ignoreTrailingWhiteSpace=True)

shots = shots_1.union(shots_2.select(column_names.tolist()))

0 shotID
1 homeTeamCode
2 awayTeamCode
3 season
4 isPlayoffGame
5 game_id
6 homeTeamWon
7 id
8 time
9 timeUntilNextEvent
10 timeSinceLastEvent
11 period
12 team
13 location
14 event
15 goal
16 shotPlayContinuedOutsideZone
17 shotPlayContinuedInZone
18 shotGoalieFroze
19 shotPlayStopped
20 shotGeneratedRebound
21 homeTeamGoals
22 awayTeamGoals
23 xCord
24 yCord
25 xCordAdjusted
26 yCordAdjusted
27 shotAngle
28 shotAngleAdjusted
29 shotAnglePlusRebound
30 shotAngleReboundRoyalRoad
31 shotDistance
32 shotType
33 shotOnEmptyNet
34 shotRebound
35 shotAnglePlusReboundSpeed
36 shotRush
37 speedFromLastEvent
38 lastEventxCord
39 lastEventyCord
40 distanceFromLastEvent
41 lastEventShotAngle
42 lastEventShotDistance
43 lastEventCategory
44 lastEventTeam
45 homeEmptyNet
46 awayEmptyNet
47 homeSkatersOnIce
48 awaySkatersOnIce
49 awayPenalty1TimeLeft
50 awayPenalty1Length
51 homePenalty1TimeLeft
52 homePenalty1Length
53 playerPositionThatDidEvent
54 playerNumThatDidEvent
55 playerNumThatDidLastEven

I would like a single variable to represent the skater situation for each shot. That is, does the team shooting the puck have a man advantage, or are they penalized and thus are down a skater? Intuitively, a team that has more skaters will have more control of the play, and will be more likely to score. Conversely, a team with fewer men will generally take lower quality shoty and thus be less likely to score. The *skaterDifference* variable accounts for those situations. 

In addition to the *skaterDifference* variable, I use several variables that capture situations where a team may be more likely to score. Below, I list the variables I will use and explain the intuition behind their inclusion.

| Variable | Intuition |
|-|-|
| isHomeTeam                | Home team might be better able to score goals|
| arenaAdjustedShotDistance | Closer shots are more likely to be scored|
| adjustedShotAngle         | Shots from sharp angles are less likely to be scored|
| shotRush                  | Shots designated as rush occur during transition and thus when defensive structure of conceding team is suboptimal|
| shotRebound               | A shot designated as a rebound is likely to occur when a goalie is out of position or has his field of vision towards the puck obstructed|
| shotAnglePlusReboundSpeed | A shot that occurs after a very quick change in the angle of its previous location may mean that the goaltender was forced to make a quick movement to adjust, which gives a higher chance to scoring|
| shooterTimeOnIce          | A player that has been on ice for long may be too tired to play at their best/shoot accurately|
| averageRestDifference     | If the shooting team is more rested than the defending team, it's more likely the latter will commit defensive lapses|
| speedFromLastEvent        | Situations where the puck moves quickly from one location to another could mean sudden movement across the ice or into the offensive zone|
| playerEncoder             | The position of the shooting player (C,L,R,D) |
| shotTypeEncoder           | Different shot types may have different likelihood of being scored|
| lastEventEncoder          | Depending on the game situation just before the event occurred, there might be events that lead to a higher probability of a goal|
| gameSituationEncoder      | Shots made under different situations have different probabilities of being scored|

In [3]:
# create columns with number of skaters for each time at time of shot attempt
shots = shots.withColumn('shootingTeamSkaters',(shots.shootingTeamForwardsOnIce.cast(IntegerType())+
                         shots.shootingTeamDefencemenOnIce.cast(IntegerType())))
shots = shots.withColumn('shootingTeamSkaters',(shots.shootingTeamSkaters.cast(StringType())))

shots = shots.withColumn('defendingTeamSkaters',(shots.defendingTeamForwardsOnIce.cast(IntegerType())+
                         shots.defendingTeamDefencemenOnIce.cast(IntegerType())))
shots = shots.withColumn('defendingTeamSkaters',(shots.defendingTeamSkaters.cast(StringType())))


# use to create a variable for the team's strength at the time of the shot
# shot from a 5v4 team different from one from 4v5 team
shots = shots.withColumn('gameSituation',func.concat(shots.shootingTeamSkaters,
                                                     func.lit('v'),
                                                     shots.defendingTeamSkaters))

full_features_list = ['isHomeTeam','arenaAdjustedShotDistance','shotAngleAdjusted','shotRush',
                 'shotRebound','shotAnglePlusReboundSpeed','shooterTimeOnIce',
                 'averageRestDifference','speedFromLastEvent','playerEncoder',
                 'shotTypeEncoder','lastEventEncoder','gameSituationEncoder']

# convert data type for most variables to required data type
for col in features_list[1:-4]:
    shots = shots.withColumn(col,shots[str(col)].cast(DecimalType()))

shots = shots.withColumn('goal',shots.goal.cast(IntegerType()))
shots = shots.withColumn('isHomeTeam',shots.isHomeTeam.cast(IntegerType()))
shots = shots.withColumn('season',shots.season.cast(IntegerType()))
    
# drop rows where we don't know the position of the player 
# who made the shot; relatively very few of these so should not affect results
shots = shots.filter('playerPositionThatDidEvent IN ("L","R","C","D")')

In [4]:
# in order to be able to be able to use categorical variables in our models,
# we need to convert them to dummy variables
shots_indexer = StringIndexer(inputCol='shotType',outputCol='shotTypeIndexer',
                               handleInvalid='skip',stringOrderType='alphabetAsc')
shots_encoder = OneHotEncoderEstimator(inputCols=['shotTypeIndexer'],outputCols=['shotTypeEncoder'])

player_indexer = StringIndexer(inputCol='playerPositionThatDidEvent',outputCol='playerIndexer',
                               handleInvalid='skip',stringOrderType='alphabetAsc')
player_encoder = OneHotEncoderEstimator(inputCols=['playerIndexer'],outputCols=['playerEncoder'])

last_event_indexer = StringIndexer(inputCol='lastEventCategory',outputCol='lastEventIndexer',
                               handleInvalid='skip',stringOrderType='alphabetAsc')
last_event_encoder = OneHotEncoderEstimator(inputCols=['lastEventIndexer'],outputCols=['lastEventEncoder'])

game_situation_indexer = StringIndexer(inputCol='gameSituation',outputCol='gameSituationIndexer',
                               handleInvalid='skip',stringOrderType='alphabetAsc')
game_situation_encoder = OneHotEncoderEstimator(inputCols=['gameSituationIndexer'],outputCols=['gameSituationEncoder'])

### Evaluating Predictions

Usually regressions are evaluated by examining a measure like *R<sup>2</sup>* or RMSE. However, I elected to use a logarithmic scoring rule, specifically the logarithmic loss, to evaluate my predictions, as per [this](https://oddacious.github.io/proper_scoring_rules) great post. One of the features of using log loss as our evaluation metric is that it discourages overly confident predictions. Thus it incentivizes for weighing variables that have a significant effect on the probability of the result of interest: in this case, scoring a goal. 

Unfortunately, PySpark does not have a log loss evaluator available by default. Thus I create it below. Since we would like to *minimize* log loss, I specify that "bigger is NOT better" in the appropriate function. This ensures that our evaluator will use this criteria when we run on it on several iterations of the model. 

In [5]:
class BinaryLogLossEvaluator(Evaluator):
    def __init__(self,predictionCol='probability',labelCol='label'):
        self.predictionCol = predictionCol
        self.labelCol = labelCol

    def _get_probabilities(self,row):
        return row[1].item()
        
    def _evaluate(self,dataset):
        _udf_get_probabilities = func.udf(self._get_probabilities,FloatType())
        dataset = dataset.withColumn('prob_1',_udf_get_probabilities(dataset.select(self.predictionCol)[0]))
        
        self.log_loss = (-1) * dataset.select(func.sum(dataset.select(self.labelCol)[0] 
                                                           * func.log(dataset.prob_1)
                                                       + (1 - dataset.select(self.labelCol)[0])
                                                           * func.log(1-dataset.prob_1)
                                                      )
                                             ).collect()[0][0]
        
        return self.log_loss
    
    def isLargerBetter(self):
        return False

Now I will specify several versions of the model and pick one that seems to perform best. I use several combinations of the variables and test them over a range of possible values for the hyperparameters of the model. For the logistic regression model in PySpark, the regularization term is:

$$\alpha(\lambda(\begin{Vmatrix}\mathbf{w}\end{Vmatrix}_1)) + (1-\alpha)(\frac{\lambda}{2}\begin{Vmatrix}\mathbf{w}\end{Vmatrix}_2^2)$$

for $\alpha\in[0,1], \lambda \geq 0$. In this case, $\begin{Vmatrix}\mathbf{w}\end{Vmatrix}_1$ is the $L_1$ regularization term (Lasso regularization), and $\begin{Vmatrix}\mathbf{w}\end{Vmatrix}_2^2$ is the $L_2$ regularization term (ridge regularization). In the case where $\lambda=0$, there is no regularization. In the case where $\alpha=1$ and $\lambda > 0$, we have a lasso regression, while the case of $\alpha=0$ and $\lambda > 0$ is a ridge regression. If $0<\alpha<1$, we have a combination of the two regularization factors. I will test a range of values for $\lambda$ and $\alpha$ in my cross-validation process and determine which is the best model for each combination of features. Once the best model for each set of features has been found, I will then test the performance of each best model on the test data and select the model that performs best out-of-sample.

### Training and Cross-Validation of Several Models

Now that I set up my data and my evaluator, I can proceed with training and testing of the models. I train several models, testing various hyperparameter values through cross-validation. I use the default k=3 of the k-fold CrossValidator class of PySpark. What this means is that PySpark will divide the training dataset into 3 parts, or folds. It will train a model with a specific combination of hyperparameters in the parameter grid using 2 of those folds. It will then test its performance on the 3rd fold, calculating the evaluation metric (in our case the log loss) for the test fold. It will repeat this process twice more, using one of the other 2 folds as the testing fold. The log loss for this set of hyperparameters is the average log loss calculated for each fold. 

We do this process for each combination of $\alpha$ and $\lambda$ in the parameter grid. Once that's done, we find the model with the lowest log loss and designate it as the "best model" for a certain combination of features. This gives us the "best foot forward" model for every combination of features/type of model we are considering. At this point, in order to choose between these "best foot forward" models, we evaluate the performance of each one on the testing dataset which we have left untouched up to this point. The model that performs best on the test dataset is the model we choose as our model to use.

To make things easier for us, once I find the best hyperparameter for each combination of features, I will evaluate that model on the testing data and print that value. We can compare those values at the end to determine the appropriate model. 

### Model Group 1: All Features

For this set of models, I am testing the optimal hyperparameters for a model which contains all the features we listed above.

In [6]:
vector_assembler = VectorAssembler(inputCols=full_features_list,outputCol='features')
pipeline = Pipeline(stages=[shots_indexer,shots_encoder,
                            player_indexer, player_encoder,
                            last_event_indexer,last_event_encoder,
                            game_situation_indexer,game_situation_encoder,
                            vector_assembler])

piped_data = pipeline.fit(shots).transform(shots)
training = piped_data.filter('season < 2018')
test = piped_data.filter('season >= 2018')
# lr = LogisticRegression(labelCol='goal')
# evaluator = BinaryLogLossEvaluator(labelCol='goal')
# grid = (ParamGridBuilder()
#            .addGrid(lr.regParam,[0,5e-4,0.1,10])
#            .addGrid(lr.elasticNetParam,[0,0.25,0.75,1])
#            .build())
# cv = CrossValidator(estimator=lr,
#                    estimatorParamMaps=grid,
#                     evaluator=evaluator)

# models = cv.fit(training)
# full_cv_best_lr = models.bestModel
# full_cv_best_lr.write().overwrite().save('../models/full_cv_best_lr')

full_cv_best_lr = LogisticRegressionModel.load('../models/full_cv_best_lr')
test = full_cv_best_lr.transform(test)
evaluator = BinaryLogLossEvaluator(labelCol='goal')
print(evaluator.evaluate(test))

45995.77407340202


### Model Group 2:

Same as the above but we exclude the **playerEncoder** variable. 

In [58]:
alt1_features_list = ['isHomeTeam','arenaAdjustedShotDistance','shotAngleAdjusted','shotRush',
                 'shotRebound','shotAnglePlusReboundSpeed','shooterTimeOnIce',
                 'averageRestDifference','speedFromLastEvent',
                 'shotTypeEncoder','lastEventEncoder','gameSituationEncoder']


vector_assembler = VectorAssembler(inputCols=alt1_features_list,outputCol='features')
pipeline = Pipeline(stages=[shots_indexer,shots_encoder,
                            player_indexer, player_encoder,
                            last_event_indexer,last_event_encoder,
                            game_situation_indexer,game_situation_encoder,
                            vector_assembler])

piped_data = pipeline.fit(shots).transform(shots)
training = piped_data.filter('season < 2018')
test = piped_data.filter('season >= 2018')

# lr = LogisticRegression(labelCol='goal')
# evaluator = BinaryLogLossEvaluator(labelCol='goal')
# grid = (ParamGridBuilder()
#         .addGrid(lr.regParam,[0,5e-4,0.1,10])
#         .addGrid(lr.elasticNetParam,[0,0.25,0.75,1])
#         .build())
# cv = CrossValidator(estimator=lr,
#                    estimatorParamMaps=grid,
#                     evaluator=evaluator)

# models = cv.fit(training)
# alt1_cv_best_lr = models.bestModel
# alt1_cv_best_lr.write().overwrite().save('../models/alt1_cv_best_lr')

alt1_cv_best_lr = LogisticRegressionModel.load('../models/alt1_cv_best_lr')

test = alt1_cv_best_lr.transform(test)
evaluator = BinaryLogLossEvaluator(labelCol='goal')
print(evaluator.evaluate(test))

45997.73713993364


### Model Group 3:

This time, we're excluding both the **playerEncoder** and **shotTypeEncoder** variables.

In [59]:
alt2_features_list = ['isHomeTeam','arenaAdjustedShotDistance','shotAngleAdjusted','shotRush',
                 'shotRebound','shotAnglePlusReboundSpeed','shooterTimeOnIce',
                 'averageRestDifference','speedFromLastEvent','lastEventEncoder','gameSituationEncoder']


vector_assembler = VectorAssembler(inputCols=alt2_features_list,outputCol='features')
pipeline = Pipeline(stages=[shots_indexer,shots_encoder,
                            player_indexer, player_encoder,
                            last_event_indexer,last_event_encoder,
                            game_situation_indexer,game_situation_encoder,
                            vector_assembler])

piped_data = pipeline.fit(shots).transform(shots)
training = piped_data.filter('season < 2018')
test = piped_data.filter('season >= 2018')

# lr = LogisticRegression(labelCol='goal')
# evaluator = BinaryLogLossEvaluator(labelCol='goal')
# grid = (ParamGridBuilder()
#         .addGrid(lr.regParam,[0,5e-4,0.1,10])
#         .addGrid(lr.elasticNetParam,[0,0.25,0.75,1])
#         .build())
# cv = CrossValidator(estimator=lr,
#                    estimatorParamMaps=grid,
#                     evaluator=evaluator)

# models = cv.fit(training)
# alt2_cv_best_lr = models.bestModel
# alt2_cv_best_lr.write().overwrite().save('../models/alt2_cv_best_lr')

alt2_cv_best_lr = LogisticRegressionModel.load('../models/alt_2_cv_best_lr')

test = alt2_cv_best_lr.transform(test)
evaluator = BinaryLogLossEvaluator(labelCol='goal')
print(evaluator.evaluate(test))

46293.58923424403


### Model Group 4:

I will revert to using all features, but I will be using a random forest classifier instead of logistic regression. 

In [62]:
# back to using the full list of features for this model
vector_assembler = VectorAssembler(inputCols=full_features_list,outputCol='features')
pipeline = Pipeline(stages=[shots_indexer,shots_encoder,
                            player_indexer, player_encoder,
                            last_event_indexer,last_event_encoder,
                            game_situation_indexer,game_situation_encoder,
                            vector_assembler])

piped_data = pipeline.fit(shots).transform(shots)
training = piped_data.filter('season < 2018')
test = piped_data.filter('season >= 2018')

# rf = RandomForestClassifier(labelCol='goal', seed=1)
# evaluator = BinaryLogLossEvaluator(labelCol='goal')
# grid = (ParamGridBuilder()
#            .addGrid(rf.maxDepth,[3,4,5,6,7,8])
#            .addGrid(rf.numTrees,[10,20,30])
#            .build())
# cv = CrossValidator(estimator=rf,
#                    estimatorParamMaps=grid,
#                     evaluator=evaluator)
# models = cv.fit(training)
# rf_cv_best_lr = models.bestModel
# rf_cv_best_lr.write().overwrite().save('../models/rf_cv_best_lr')

rf_cv_best_lr = RandomForestClassificationModel.load('../models/rf_cv_best_lr')

test = rf_cv_best_lr.transform(test)
evaluator = BinaryLogLossEvaluator(labelCol='goal')
print(evaluator.evaluate(test))

47764.83686744264


### Result and Sanity Check

I find that the logistic regression model with all my variables performs best. I should note however that the model that excludes shooting player position (centre, winger, or defenseman) performs almost exactly as well. This implies that a player's position has almost no bearing on the probability of their scoring of a goal. What this means is once we control for different variables that affect shot quality, we can conclude that shooting talent is on average the same across these different player groups. This emphasizes the importance of shot quality and selection in scoring. The model implies that the difference in goal scoring between the average defenseman and the average forward can be attributed in large part to shot quantity and quality. 

Let's do a sanity check. Who were the best performing players at normal play (i.e. 5 on 5) of the 2018-2019 season?

In [7]:
def get_xGoal(row):
    return row[1].item()

udf_get_xGoal = func.udf(get_xGoal,DoubleType())

# recreating the data pipeline to be the one used in the first model again
# to redo the predictions for that model
vector_assembler = VectorAssembler(inputCols=full_features_list,outputCol='features')
pipeline = Pipeline(stages=[shots_indexer,shots_encoder,
                            player_indexer, player_encoder,
                            last_event_indexer,last_event_encoder,
                            game_situation_indexer,game_situation_encoder,
                            vector_assembler])

piped_data = pipeline.fit(shots).transform(shots)
training = piped_data.filter('season < 2018')
test = piped_data.filter('season >= 2018')

test = full_cv_best_lr.transform(test)
test = test.withColumn('xGoalPred',udf_get_xGoal(test.probability))
test = test.withColumn('season',test.season.cast(IntegerType()))

In [12]:
result = test.filter('gameSituation="5v5" AND season=2018').groupBy('shooterName','shooterPlayerId',).agg({'xGoalPred':'sum','goal':'sum'})
result = result.withColumn('sum(xGoalPred)',func.round('sum(xGoalPred)',1))
result.sort('sum(xGoalPred)',ascending=False).show(25)

+--------------------+---------------+---------+--------------+
|         shooterName|shooterPlayerId|sum(goal)|sum(xGoalPred)|
+--------------------+---------------+---------+--------------+
|          Timo Meier|        8478414|       26|          26.0|
|        John Tavares|        8475166|       34|          25.1|
|   Brendan Gallagher|        8475848|       28|          24.7|
|        Evander Kane|        8475169|       23|          23.1|
|        Cam Atkinson|        8474715|       27|          22.1|
|        Tyler Seguin|        8475794|       20|          21.9|
|     Auston Matthews|        8479318|       26|          21.3|
|    Nathan MacKinnon|        8477492|       26|          21.2|
|  Vladimir Tarasenko|        8475765|       24|          21.1|
|      Brayden Schenn|        8475170|       13|          20.1|
|        Jeff Skinner|        8475784|       27|          19.7|
|       Ryan O'Reilly|        8475158|       25|          19.4|
|       Sebastian Aho|        8478427|  

Depending on how familiar you are with the NHL, this list should mostly present no surprises. There are many stars on this list, and all are players who are considered to be good offensive players. Of course, one would question whether Timo Meier would qualify as the best offensive player in the league for that season, but it is not a completely unreasonable given how close his shots tend to be to the net and how many are rebounds and tip-ins. 

The difference between the xGoals and actual goals is a proxy for shooting talent: a player that consistently outperforms their expected goals measure is likely to be a better shooter than average, as they can score from more difficult situations that other players find difficult to score from. Who were these players in 2018-2019?

In [14]:
result = test.filter('gameSituation="5v5" AND season=2018').groupBy('shooterName','shooterPlayerId',).agg({'xGoalPred':'sum','goal':'sum'})

result = result.withColumn('goalsAbove',func.round(result.select('sum(goal)')[0]-result.select('sum(xGoalPred)')[0],1))
result = result.withColumn('sum(xGoalPred)',func.round('sum(xGoalPred)',1))
result.sort('goalsAbove',ascending=False).show(25)

+----------------+---------------+---------+--------------+----------+
|     shooterName|shooterPlayerId|sum(goal)|sum(xGoalPred)|goalsAbove|
+----------------+---------------+---------+--------------+----------+
|  Leon Draisaitl|        8477934|       28|          13.9|      14.1|
|   Jake Guentzel|        8477404|       32|          18.2|      13.8|
|  Alex DeBrincat|        8479337|       24|          12.5|      11.5|
|  Steven Stamkos|        8474564|       23|          11.7|      11.3|
|   Alex Ovechkin|        8471214|       29|          18.5|      10.5|
| Nikita Kucherov|        8476453|       24|          14.0|      10.0|
|    Patrick Kane|        8474141|       26|          16.2|       9.8|
|      Mark Stone|        8475913|       26|          16.5|       9.5|
|Viktor Arvidsson|        8478042|       24|          14.6|       9.4|
|    David Perron|        8474102|       19|           9.7|       9.3|
|   Morgan Rielly|        8476853|       17|           7.8|       9.2|
|     

According to my model, Leon Draisaitl and Jake Guentzel both exhibited excellent shooting talent in the 18-19 season. This makes sense when you consider that they spent most of their playing time in that season with Connor McDavid and Sidney Crosby respectively, two of the league's best passers and best overall players. Further on below, we see some familiar names: Alex Ovechkin, Steven Stamkos, and Patrick Kane.